In [1]:
##Cormac McHale

In [2]:
##Training the Neural Network

In [3]:
import gzip
import sklearn.preprocessing as pre
import keras as kr
import numpy as np
from keras.models import load_model

Using TensorFlow backend.


In [4]:
# Start a neural network, building it by layers.
model = kr.models.Sequential()

# Add a hidden layer with 1000 neurons and an input layer with 784.
model.add(kr.layers.Dense(units=1000, activation='relu', input_dim=784))
#model.add(kr.layers.Dense(units=100, activation='sigmoid'))
model.add(kr.layers.Dense(units=1000, activation='relu'))
# Add a 10 neuron output layer for each digit
model.add(kr.layers.Dense(units=10, activation='softmax'))

# Build the graph.
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [5]:
##file reading and formatting
imagesFile  = open('MNIST/train-images.idx3-ubyte', 'rb')
labelsFile = open('MNIST/train-labels.idx1-ubyte', 'rb')
images = imagesFile.read()
labels = labelsFile.read()
#getting the file info ready to train the model
images =  (~np.array(list(images[16:])).reshape(60000, 28, 28).astype(np.uint8)-255)
labels =  np.array(list(labels[ 8:])).astype(np.uint8)

In [6]:
inputs = images.reshape(60000, 784)
#formatting the data to see if we can acheieve better results
inputs[inputs>0] = 1
inputs[inputs<1] = 0 

In [7]:
#label the output layer
encoder = pre.LabelBinarizer()
encoder.fit(labels)
outputs = encoder.transform(labels)
#just for looking
#print(inputs[0])
#print (inputs[0].size)

In [8]:
#view of the model
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 1000)              785000    
_________________________________________________________________
dense_2 (Dense)              (None, 1000)              1001000   
_________________________________________________________________
dense_3 (Dense)              (None, 10)                10010     
Total params: 1,796,010
Trainable params: 1,796,010
Non-trainable params: 0
_________________________________________________________________


In [9]:
##start training the neural network
model.fit(inputs, outputs, epochs=5, batch_size=150)

Epoch 1/5
60000/60000 [==============================] - 19s 324us/step - loss: 0.2163 - accuracy: 0.9334
Epoch 2/5
60000/60000 [==============================] - 19s 314us/step - loss: 0.0902 - accuracy: 0.9713
Epoch 3/5
60000/60000 [==============================] - 19s 310us/step - loss: 0.0633 - accuracy: 0.9795s - loss: 0.0633 
Epoch 4/5
60000/60000 [==============================] - 19s 313us/step - loss: 0.0466 - accuracy: 0.9850
Epoch 5/5
60000/60000 [==============================] - 19s 312us/step - loss: 0.0362 - accuracy: 0.9884


In [10]:
##need to then use the test images on the Neural Network...
imagesTest  = open('MNIST/t10k-images.idx3-ubyte', 'rb')
labelsTest = open('MNIST/t10k-labels.idx1-ubyte', 'rb')
testImages = imagesTest.read()
testLabels = labelsTest.read()

In [11]:
#checking the file
#type(testImages)

In [12]:
#checking the data
#testImages[0:4]

In [13]:
#prepare the test images for prediction
testImages = (~np.array(list(testImages[16:])).reshape(10000, 28, 28).astype(np.uint8)-255)
testLabels =  np.array(list(testLabels[ 8:])).astype(np.uint8)
testInputs = testImages.reshape(10000, 784)
testInputs[testInputs>0] = 1
testInputs[testInputs<1] = 0 
encoder = pre.LabelBinarizer()
encoder.fit(testLabels)
testOutputs = encoder.transform(testLabels)

In [14]:
# Have the network predict the classes of the test inputs.
predictions = model.predict(testInputs)
predictions = encoder.inverse_transform(predictions)
##Comparison
predictions == encoder.inverse_transform(testOutputs)
#check how many it guessed correctly
(predictions == encoder.inverse_transform(testOutputs)).sum()

9724

In [15]:
#save the model
model.save('savedModel/my_model.h5')
##Initial Model finished... flask Next.. will return to this later
testInputs = testImages.reshape(10000, 784)
model.predict(testInputs)

array([[3.39476450e-12, 5.99654632e-11, 4.73377710e-08, ...,
        1.00000000e+00, 1.09953296e-10, 2.44869858e-09],
       [3.66995713e-12, 8.35601865e-11, 9.99999881e-01, ...,
        2.75814760e-10, 1.65799267e-11, 1.17796482e-14],
       [4.67504346e-09, 9.99861836e-01, 5.83463134e-06, ...,
        1.15644862e-05, 1.07188607e-04, 3.13949600e-09],
       ...,
       [1.43387215e-13, 2.06175271e-10, 4.09314683e-12, ...,
        1.30224116e-08, 8.67969518e-09, 6.26935787e-07],
       [7.14079490e-12, 3.90377869e-10, 1.86615176e-12, ...,
        1.03870246e-10, 1.71682200e-06, 4.39113607e-11],
       [7.31146602e-11, 7.20734375e-11, 1.99567862e-10, ...,
        4.61866656e-12, 1.76909339e-07, 1.71528924e-08]], dtype=float32)

In [16]:
##Model complete

In [17]:
##Re-train with an extra layer

In [18]:
type(model)

keras.engine.sequential.Sequential

In [19]:
##loop through old model
#and add all the trained layers to a new model and make them un-trainable
fineTune = kr.models.Sequential()
for layer in model.layers:
    fineTune.add(layer)
for layer in fineTune.layers:
    layer.trainable = False

In [20]:
#add an extra output layer
#with sigmoid activation
#refine the guess once more
fineTune.add(kr.layers.Dense(units=10, activation='sigmoid'))

In [21]:
#compile and re-train
fineTune.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
fineTune.fit(inputs, outputs, epochs=5, batch_size=50)

Epoch 1/5
60000/60000 [==============================] - 8s 132us/step - loss: 0.3992 - accuracy: 0.8921
Epoch 2/5
60000/60000 [==============================] - 8s 127us/step - loss: 0.1554 - accuracy: 0.9839
Epoch 3/5
60000/60000 [==============================] - 8s 125us/step - loss: 0.0824 - accuracy: 0.9976
Epoch 4/5
60000/60000 [==============================] - 8s 128us/step - loss: 0.0489 - accuracy: 0.9978
Epoch 5/5
60000/60000 [==============================] - 8s 130us/step - loss: 0.0311 - accuracy: 0.9979


In [22]:
# Have the network predict the classes of the test inputs.
predictions = fineTune.predict(testInputs)
predictions = encoder.inverse_transform(predictions)
##Comparison
predictions == encoder.inverse_transform(testOutputs)
#check how many it guesses correctly
(predictions == encoder.inverse_transform(testOutputs)).sum()

9725

In [23]:
#Results
#save new model
fineTune.save('savedModel/freshlyTrained.h5')
#test in client